In [1]:
import json
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import pandas as pd

/home/MERHAWI/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

with open('document_with_ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [4]:
es_client = Elasticsearch("http://localhost:9200")

In [5]:
es_client.info()

ObjectApiResponse({'name': '02dbdefb68b9', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'tTU4JnvRRqyY4mzgX2u84Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:

index_settings = {
    "settings": {
        "number_of_shards" : 1,
        "number_of_replicas" : 0
    },
    "mappings" : {
        "properties" : {
            "text" : {"type" : "text"},
            "section" : {"type" : "text"},
            "question" : {"type" : "text"},
            "course" : {"type" : "keyword"},
            "id" : {"type" : "keyword"},
        }
    }
}

index_name = "course_questions"
es_client.indices.delete(index = index_name, ignore_unavailable=True)
es_client.indices.create(index = index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [7]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████| 948/948 [00:06<00:00, 151.91it/s]


In [8]:
def elastic_search(query,course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index= index_name, body= search_query)  
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        
    return result_docs

In [9]:
elastic_search(
    query="I just discovered the course. Can I still join?",
    course='data-engineering-zoomcamp'
)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'id': '7842b56a'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp',
  'id': '63394d91'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it fin

In [10]:
df_ground_truth = pd.read_csv('ground_truth_data.csv')

In [11]:
df_ground_truth

,question,course,document
0,What is the starting date and time for the cou...,data-engineering-zoomcamp,c02e79ef
1,How do I subscribe to the course calendar?,data-engineering-zoomcamp,c02e79ef
2,Where can I find the registration link for the...,data-engineering-zoomcamp,c02e79ef
3,Is there a Telegram channel for course announc...,data-engineering-zoomcamp,c02e79ef
4,Do I need to register in DataTalks.Club's Slack?,data-engineering-zoomcamp,c02e79ef
...,...,...,...
4730,What steps should I take to remove infrastruct...,mlops-zoomcamp,886d1617
4731,Can you explain the process for destroying AWS...,mlops-zoomcamp,886d1617
4732,What commands do I need to run locally to dest...,mlops-zoomcamp,886d1617
4733,Is there a specific backend configuration I ne...,mlops-zoomcamp,886d1617


In [12]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [13]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = elastic_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|████████████████████████████████████████████████████████████████████████████| 4735/4735 [00:20<00:00, 230.86it/s]


In [14]:
example_metrix = [
    [False, False, False, False, False],
    [False, False, False, False, False],
    [False, False, False, False, False],
    [False, True, False, False, False],
    [False, False, False, False, False],
    [True, False, False, False, False],
    [False, False, False, True, False],
    [True, False, False, False, False],
    [True, False, False, False, False],
    [False, True, False, False, False],
    [False, False, False, False, True],
    [False, False, False, False, True]
]

- Hit Rate
- MRR (Mean Reciprocal Rank)

In [15]:
def hit_rate(relavance_total):
    cnt = 0
    for line in relavance_total:
        if True in line:
            cnt = cnt + 1
    return cnt / len(relavance_total)

In [16]:
def mrr_score(relavance_total):
    score = 0.0
    for line in relavance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                score = score + 1 / (rank + 1)
        
    return score / len(relavance_total)

In [17]:
hit_rate(relevance_total),mrr_score(relevance_total)

(0.6378035902851109, 0.4952692713833152)

In [18]:
import minsearch

index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course", "id"]
)

index.fit(documents)

In [19]:
def minsearch_search(query, course):
    boost = {'question': 3.0, 'section':0.5}

    results = index.search(
        query = query,
        filter_dict = {'course': course},
        boost_dict = boost,
        num_results = 5
    )

    return results

In [20]:
relevance_total_minsearch = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = minsearch_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total_minsearch.append(relevance)

100%|████████████████████████████████████████████████████████████████████████████| 4735/4735 [00:15<00:00, 314.39it/s]


In [21]:
hit_rate(relevance_total_minsearch), mrr_score(relevance_total_minsearch)

(0.7157338965153115, 0.5870362548398453)

In [22]:
0.7157338965153115-0.6378035902851109

0.07793030623020059

In [23]:
def evaluation_function(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)
    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr_score': mrr_score(relevance_total)
    }

In [24]:
evaluation_function(ground_truth, lambda q: minsearch_search(q['question'], course=q['course']))

100%|████████████████████████████████████████████████████████████████████████████| 4735/4735 [00:15<00:00, 309.65it/s]


{'hit_rate': 0.7157338965153115, 'mrr_score': 0.5870362548398453}

In [25]:
evaluation_function(ground_truth, lambda q: elastic_search(q['question'], course=q['course']))

100%|████████████████████████████████████████████████████████████████████████████| 4735/4735 [00:18<00:00, 261.09it/s]


{'hit_rate': 0.6378035902851109, 'mrr_score': 0.4952692713833152}